## Heart disease prediction - Model evaluation using nested cross validation
In this notebook my goal is to evaluate different types of classifiers to see which type performs the best in predicting heart disease in patients based on a number of attributes. To do this I will perform nested cross validation on the full dataset to evaluate the generalized performance of the models. The data comes from [Kaggle and UCI](https://www.kaggle.com/ronitf/heart-disease-uci). 

In [49]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
#matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Load the data

In [50]:
data = pd.read_csv("data/heart.csv")

Let's look at the data:

In [74]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [51]:
data.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

Columns can be described as such:

**age** - age in years  
**sex** - (1 = male; 0 = female)  
**cp** - chest pain type  
**trestbps** - resting blood pressure (in mm Hg on admission to the hospital)  
**chol** - serum cholestoral in mg/dl  
**fbs** - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)  
**restecg** - resting electrocardiographic results  
**thalach** - maximum heart rate achieved  
**exang** - exercise induced angina (1 = yes; 0 = no)  
**oldpeak** - ST depression induced by exercise relative to rest  
**slope** - the slope of the peak exercise ST segment  
**ca** - number of major vessels (0-3) colored by flourosopy  
**thal** - 3 = normal; 6 = fixed defect; 7 = reversable defect  
**target** - 1 or 0 (Whether heart disease is present in patient or not)  

As we can see in the report above, the target variable is balanced between positive and negative in the dataset. There are no null values, and for the categorical categories, we need to create dummy variables to remove the linear relation between them. First, let's split the data into X and y:

In [52]:
X = data.drop("target", axis=1)
y= data["target"]

### Process the data
After loading the data we can create dummy features for each of the categorical columns:

In [53]:
ca_dummies = pd.get_dummies(X["ca"], prefix="ca")
cp_dummies = pd.get_dummies(X["cp"], prefix="cp")
rest_ecg_dummies = pd.get_dummies(X["restecg"], prefix="rest_ecg")
slope_dummies = pd.get_dummies(X["slope"], prefix="slope")
thal_dummies = pd.get_dummies(X["thal"], prefix="thal")

And drop the old columns and merge:

In [54]:
X = X.drop(["ca", "cp", "restecg", "slope", "thal"], axis=1)

In [55]:
X = pd.concat((X, ca_dummies, cp_dummies, rest_ecg_dummies, slope_dummies, thal_dummies), axis=1)

Let's also create a normalized version of the features, scaling all of them to 0-1:

In [56]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
columns = X.columns
X_norm = pd.DataFrame(scaler.fit_transform(X), columns = columns)

### Model evaluation
Let's now get into the model evaluation. I want to test four different classifiers: Random Forest, SVC, LogisticRegression, and RidgeClassifier.

In [57]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier



To evaluate each classifiers general performance on our data without the risk of overfitting on our test set, I will use nested crossvalidation. Normally, we use cross validation with GridSearchCV to find the best hyperparameters for our model. In nested cross validation, we will do such a grid search and find the best score on the validation set for an "outer" 80-20 cross validation split. For a better explanation, see [here](https://sebastianraschka.com/faq/docs/evaluate-a-model.html). 

Since the target variable is balanced but the dataset is small enough that for cross validation it may not be balanced, and most of the classifiers are not made to return probabilities (which makes roc_auc score not optimal), I will use balanced accuracy as the metric. Let's define a function which takes in a classifier, parameter, and data, and returns a generalized score:

In [58]:
def generalize_score(estimator, params, X_train, y_train):
    
    inner_kf = KFold(n_splits=5, random_state=10, shuffle=True)
    
    outer_kf = KFold(n_splits=5, random_state = 20, shuffle=True)

    clf = GridSearchCV(estimator = estimator, param_grid=params, cv=inner_kf)
    
    nested_score = cross_val_score(clf, X=X_train, y=y_train, scoring="balanced_accuracy", cv = outer_kf)
    
    return nested_score.mean()



Now for the models:

## Random forest

In [59]:
# RandomForest

rf = RandomForestClassifier()
params = {'criterion': ['entropy', 'gini'],
          'max_depth': np.arange(2,11, 2),
          'min_samples_leaf': np.arange(1,40, 10),
          'min_samples_split': np.arange(2, 100, 10),
          'n_estimators': [5, 10, 20]}


rf_score = generalize_score(rf, params, X, y)
rf_score


0.798908359502181

## SVC

In [60]:
# SVC

svc = SVC()

params = {'kernel': ['rbf'],
          'C': np.logspace(-5,5, 10),
          'gamma': np.logspace(-3,0, 3),}

svc_score = generalize_score(svc, params, X, y)

svc_score

0.6385995962677885

Since the regularization for SVCs are affected by the feature scaling, let's see if we can get a better result with normalized features:

In [61]:
# SVC Normalized

# SVC

svc = SVC()

params = {'kernel': ['rbf'],
          'C': np.logspace(-5,5, 10),
          'gamma': np.logspace(-3,0, 3),}

svc_score_norm = generalize_score(svc, params, X_norm, y)

svc_score_norm

0.8304219172216885

The result is much better!

## Logistic Regression

In [65]:
# Logistic Regression Not normalized

lr = LogisticRegression()

params = {
          'C': np.logspace(-10,10, 100),
          'solver': ["liblinear"]
          }

lr_score = generalize_score(lr, params, X, y)
lr_score

0.8331354561194377

Let's try the same thing here as for SVC, with normalized data:

In [66]:
# Logistic Regression Normalized

lr = LogisticRegression()

params = {
          'C': np.logspace(-10,10, 100),
          'solver': ["liblinear"]
          }

lr_score_norm = generalize_score(lr, params, X_norm, y)
lr_score_norm

0.8270748500588316

The results are actually a little bit worse this time.

## Ridge classifier

In [67]:
# RidgeClassifier

rc = RidgeClassifier()

params = {
          'alpha': np.logspace(-10,10, 100),
          'solver': ["auto"]
          }

rc_score = generalize_score(rc, params, X_norm, y)
rc_score

0.831789755460236

In [68]:
# RidgeClassifier Normalized


rc = RidgeClassifier()

params = {
          'alpha': np.logspace(-10,10, 100),
          'solver': ["auto"]
          }

rc_score_norm = generalize_score(rc, params, X_norm, y)
rc_score_norm

0.831789755460236

### Results & Conclusion
Let's compared the classifiers by putting them into a table.

In [69]:
results = pd.DataFrame({"Classifier": ["Random Forest", 
                                       "SVC", 
                                       "SVC Normalized", 
                                       "Logistic Regression", 
                                       "Logistic Regression Normalized", 
                                       "Ridge Classifier", 
                                       "Ridge Classifier Normalized"],
                       "Generalized accuracy score": [rf_score,
                                                     svc_score,
                                                     svc_score_norm,
                                                     lr_score,
                                                     lr_score_norm,
                                                     rc_score,
                                                     rc_score_norm]})
    
results.sort_values("Generalized accuracy score", ascending=False)

,Classifier,Generalized accuracy score
3,Logistic Regression,0.833135
5,Ridge Classifier,0.831790
6,Ridge Classifier Normalized,0.831790
2,SVC Normalized,0.830422
4,Logistic Regression Normalized,0.827075
0,Random Forest,0.798908
1,SVC,0.638600


As we can see in the table above, Logistic Regression performs slightly better than the other algorithms, with the only outlier being SVC without normalized features. Let's fit a model using LR and look at the results. This time we will use cross validation and grid search to find the best hyperparameters. To validate our results, we can run 20 iterations of splitting into train and test and running cross validation to get the best parameters, and check the average balanced accuracy on the test set. This is similar to what we did above for all models, with the difference being that we are splitting into training and test randomly instead of using KFold.

In [72]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

accuracys = []
lr = LogisticRegression()

params = {
          'C': np.logspace(-10,10, 100),
          'solver': ["liblinear"]
          }
for i in range(20):
    X_tr, X_te, y_tr, y_te = train_test_split(X,y, test_size = 0.2)

    cv = KFold(n_splits=5, shuffle=True)
    clf = GridSearchCV(lr, param_grid=params, scoring="balanced_accuracy", cv=cv)
    clf.fit(X_tr, y_tr)
    best = clf.best_estimator_
    y_pred = best.predict(X_te)

    accuracy = balanced_accuracy_score(y_te, y_pred)
    
    accuracys.append(accuracy)


In [73]:
np.mean(accuracys)

0.8500685594024592

So the average balanced accuracy we get is 85%, which is similar to what we got above.